In [1]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
# from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
# import numpy as np
# import sklearn
import pickle
from keras.applications import *
from keras.preprocessing.image import *
from keras.losses import mse, binary_crossentropy
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import random,math
from tqdm import tqdm ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping
from sklearn.semi_supervised import LabelSpreading

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# mean = np.load('feature/fbank2/mean.npy')
# std = np.load('feature/fbank2/std.npy')
min_ = np.load('feature/fbank2/min.npy')
range_ = np.load('feature/fbank2/range.npy')
X_test = np.load('feature/fbank2/X_test.npy')
X_train = np.load('feature/fbank2/X_train.npy')
X_train = np.concatenate((X_train,X_test))
# X_train = (X_train - mean)/std
# X_train = (X_train - min_)/range_
X_train.shape

(18873, 128, 63, 1)

## VAE Model

In [3]:
# parameters
laten_dim = 32
ker_si = 2
filters = 32
loss_fn = 'mse'

In [4]:
# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# then z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [5]:
input_img = Input(shape=(X_train.shape[1],X_train.shape[2],1))

x = Conv2D(filters,kernel_size=ker_si,padding='same')(input_img)
# x = Conv2D(filters*2,(2,2),padding='same',activation='relu')(x)
# x = Conv2D(filters*4,(2,2),padding='same',activation='relu')(x)
shape = K.int_shape(x)

x = Flatten()(x)
x = Dense(laten_dim*32,activation='selu')(x)
z_mean = Dense(laten_dim,name='z_mean')(x)
z_log_var = Dense(laten_dim,name='z_log_var')(x)
z = Lambda(sampling, output_shape=(laten_dim,),name='z')([z_mean,z_log_var])

encoder = Model(input_img,[z_mean,z_log_var,z],name='encoder')
encoder.summary()

latent_inputs = Input(shape=(laten_dim,), name='z_sampling')
x = Dense(shape[1] * shape[2] * shape[3], activation='selu')(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3]))(x)

# x = Conv2DTranspose(filters=filters*4,kernel_size=ker_si,activation='relu',padding='same')(x)
# x = Conv2DTranspose(filters=filters*2,kernel_size=ker_si,activation='relu',padding='same')(x)
x = Conv2DTranspose(filters=filters,kernel_size=ker_si,activation='selu',padding='same')(x)
outputs = Conv2DTranspose(filters=1,kernel_size=ker_si,activation='sigmoid',padding='same',name='decoder_output')(x)
decoder = Model(latent_inputs,outputs,name='decoder')
decoder.summary()

outputs = decoder(encoder(input_img)[2])
vae = Model(input_img,outputs,name='vae')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 63, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 63, 32)  160         input_1[0][0]                    
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 258048)       0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1024)         264242176   flatten_1[0][0]                  
__________________________________________________________________________________________________
z_mean (De

In [6]:
if loss_fn == 'mse':
    reconstruction_loss = mse(K.flatten(input_img), K.flatten(outputs))
else:
    reconstruction_loss = binary_crossentropy(K.flatten(input_img),K.flatten(outputs))
# reconstruction_loss *= X_train.shape[1] * X_train.shape[2]
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer=Nadam())
vae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 63, 1)        0         
_________________________________________________________________
encoder (Model)              [(None, 32), (None, 32),  264307936 
_________________________________________________________________
decoder (Model)              (None, 128, 63, 1)        8519841   
Total params: 272,827,777
Trainable params: 272,827,777
Non-trainable params: 0
_________________________________________________________________


In [7]:
vae.fit(X_train,epochs=50 ,shuffle=True,batch_size=64)
# vae.fit_generator(batch_generator(X_train,X_train,2),samples_per_epoch=len(X_train)//2)

Epoch 1/50
 1088/18873 [>.............................] - ETA: 2:10 - loss: nan

KeyboardInterrupt: 

In [ ]:
def generate_batch_data_random(x, y, batch_size):
    """逐步提取batch数据到显存，降低对显存的占用"""
    ylen = len(y)
    loopcount = ylen // batch_size
    while (True):
        i = random.randint(0,loopcount)
        yield x[i * batch_size:(i + 1) * batch_size]

In [ ]:
def batch_generator(X_gen,Y_gen,batch_size):
        while True: 
          yield(X_gen[batch_size],Y_gen[batch_size])

## CNN Autoencoder Model

In [40]:
input_img = Input(shape=(X_train.shape[1],X_train.shape[2],1))

x = Conv2D(64,(4,7),activation='selu',padding='same')(input_img)
x = Conv2D(32,(4,7),activation='selu',padding='same')(x)
x = Conv2D(1,(4,7),activation='selu',padding='same')(x)
shape3 = K.int_shape(x)
# x = MaxPool2D((4,11))(x)
# x = Dropout(0.25)(x)
shape = K.int_shape(x)
x = Flatten()(x)
encoder = Dense(int(shape[1]*shape[2]*shape[3]),activation='selu')(x)

# x = Dense(int(shape[1]*shape[2]*shape[3]/5),activation='relu')(x)
# x = Dropout(0.25)(x)
# encoder = Dense(shape[1]*shape[2],activation='relu')(x)
# x = Dense(int(shape[1]*shape[2]*shape[3]/5),activation='relu')(encoder)
# x = Dropout(0.25)(x)
# x = Dense(int(shape[1]*shape[2]*shape[3]),activation='relu')(x)
# x = Dropout(0.25)(x)
# x = Reshape((shape[1],shape[2],shape[3]))(x)

x = Reshape((shape[1],shape[2],shape[3]))(encoder)
# decoder = UpSampling2D((4,11))(x)
decoder = Conv2DTranspose(64,(4,7),padding='same',activation='selu')(x)
decoder = Conv2DTranspose(1,(4,7),padding='same',activation='linear')(decoder)
shape2 = K.int_shape(decoder)
autoencoder = Model(input_img,decoder)
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 128, 63, 1)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 128, 63, 64)       1856      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 128, 63, 32)       57376     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 128, 63, 1)        897       
_________________________________________________________________
flatten_3 (Flatten)          (None, 8064)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 8064)              65036160  
_________________________________________________________________
reshape_3 (Reshape)          (None, 128, 63, 1)        0         
__________

In [38]:
X_train

array([[[[0.00000000e+00],
         [0.00000000e+00],
         [0.00000000e+00],
         ...,
         [0.00000000e+00],
         [0.00000000e+00],
         [0.00000000e+00]],

        [[0.00000000e+00],
         [0.00000000e+00],
         [0.00000000e+00],
         ...,
         [0.00000000e+00],
         [0.00000000e+00],
         [0.00000000e+00]],

        [[0.00000000e+00],
         [0.00000000e+00],
         [0.00000000e+00],
         ...,
         [0.00000000e+00],
         [0.00000000e+00],
         [0.00000000e+00]],

        ...,

        [[2.68515982e-08],
         [6.10931256e-11],
         [1.00922095e-11],
         ...,
         [4.49030556e-13],
         [6.76089206e-12],
         [9.87279343e-13]],

        [[3.24172773e-08],
         [4.59462553e-11],
         [6.33367303e-12],
         ...,
         [7.08938122e-13],
         [6.00601911e-12],
         [1.19238179e-12]],

        [[2.60941008e-08],
         [1.63177358e-12],
         [3.09327118e-12],
         ...,
 

In [4]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(X_train,X_train)

Epoch 1/1
18873/18873 [==============================] - 10730s 569ms/step - loss: 4.3904e-06


In [41]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=10, min_lr=1e-4)
# autoencoder.compile(optimizer=Nadam(), loss='mse')
autoencoder.compile(optimizer=Adam(lr=0.002), loss='mse')
autoencoder.fit(X_train,X_train, epochs=500,shuffle=True,batch_size=256,callbacks=[reduce_lr])

Epoch 1/500
18873/18873 [==============================] - 33s 2ms/step - loss: 0.0313
Epoch 2/500
18873/18873 [==============================] - 33s 2ms/step - loss: 0.0200
Epoch 3/500
18873/18873 [==============================] - 33s 2ms/step - loss: 0.0138
Epoch 4/500
18873/18873 [==============================] - 33s 2ms/step - loss: 0.0105
Epoch 5/500
18873/18873 [==============================] - 33s 2ms/step - loss: 0.0085
Epoch 6/500
18873/18873 [==============================] - 33s 2ms/step - loss: 0.0068
Epoch 7/500
18873/18873 [==============================] - 33s 2ms/step - loss: 0.0048
Epoch 8/500
18873/18873 [==============================] - 33s 2ms/step - loss: 0.0040
Epoch 9/500
18873/18873 [==============================] - 33s 2ms/step - loss: 0.0035
Epoch 10/500
18873/18873 [==============================] - 33s 2ms/step - loss: 0.0029
Epoch 11/500
18873/18873 [==============================] - 33s 2ms/step - loss: 0.0022
Epoch 12/500
18873/18873 [===============

KeyboardInterrupt: 

In [8]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=2e-4)
autoencoder.compile(optimizer=Adamax(decay=1e-6), loss='mse')
autoencoder.fit(X_train,X_train, epochs=500,shuffle=True,batch_size=256,callbacks=[reduce_lr])

Epoch 1/500
18873/18873 [==============================] - 32s 2ms/step - loss: 62.9312
Epoch 2/500
18873/18873 [==============================] - 32s 2ms/step - loss: 1.5340
Epoch 3/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.9289
Epoch 4/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.6724
Epoch 5/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.5248
Epoch 6/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.4282
Epoch 7/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.3583
Epoch 8/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.3062
Epoch 9/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.2658
Epoch 10/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.2336
Epoch 11/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.2077
Epoch 12/500
18873/18873 [==============

18873/18873 [==============================] - 32s 2ms/step - loss: 0.0584
Epoch 95/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.0565
Epoch 96/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.0512
Epoch 97/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.0540
Epoch 98/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.0539
Epoch 99/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.0518
Epoch 100/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.0565
Epoch 101/500
18873/18873 [==============================] - 32s 2ms/step - loss: 0.0510
Epoch 102/500
  256/18873 [..............................] - ETA: 31s - loss: 0.0421

KeyboardInterrupt: 

In [9]:
autoencoder.save('model/lgd_dense_AE3.h5')
# encoder.save('model/lgd_dense_enc.h5')

## Label Spreading

In [11]:
# map_path = path of 'map.pkl'
map_path = 'data/map.pkl'#sys.argv[1]

# base_path = directory of all the data (train_label.csv, X_train.npy)
base_path = 'feature/fbank2/'#sys.argv[2]


with open(map_path, 'rb') as f:
    map_dict = pickle.load(f)

verified = []
unverified = []

train_label_path = os.path.join(base_path, 'train_label.csv')
Y_train = pd.read_csv(train_label_path)

for i in range(len(Y_train)):
    if Y_train['manually_verified'][i] == 1:
        verified.append(i)
    else:
        unverified.append(i)

Y_un = Y_train.loc[unverified,:]

fname_all = Y_un['fname']
fname_all = np.array(fname_all)

Y_dict = Y_un['label'].map(map_dict)
Y_dict = np.array(Y_dict)

Y_all = []
for i in Y_dict:
    Y_all.append(to_categorical(i, num_classes=41))
Y_all = np.array(Y_all)

filename = os.path.join(base_path, 'fname_unverified.npy')
np.save(filename, fname_all)

filename = os.path.join(base_path, 'y_unverified.npy')
np.save(filename, Y_all)

X_train_path = os.path.join(base_path, 'X_train.npy')
X_all = np.load(X_train_path)

X_un = X_all[unverified, :]
filename = os.path.join(base_path, 'X_unverified.npy')
np.save(filename, X_un)

X_ver = X_all[verified, :]
train_label_path = os.path.join(base_path, 'train_label.csv')
Y_ver = pd.read_csv(train_label_path)
Y_ver = Y_ver.loc[verified,:]
'''
exit()
X_all = X_all[verified, :]
idx = list(range(X_all.shape[0]))
random.shuffle(idx)
xSize = math.ceil(X_all.shape[0] / num_fold)
split_X_path = os.path.join(base_path, 'X')
split_y_path = os.path.join(base_path, 'y')
if not os.path.exists(split_X_path):
    os.makedirs(split_X_path)
if not os.path.exists(split_y_path):
    os.makedirs(split_y_path)
for i in range(num_fold):
    X = X_all[idx[i*xSize:i*xSize+xSize]]
    y = Y_all[idx[i*xSize:i*xSize+xSize]]
    print('Saving fold {}'.format(i+1))
    print('X.shape:', X.shape)
    print('y.shape:', y.shape)
    filename = os.path.join(split_X_path, 'X' + str(i+1) + '.npy')
    np.save(filename, X)
    filename = os.path.join(split_y_path, 'y' + str(i+1) + '.npy')
    np.save(filename, y)
    time.sleep(1)
'''

"\nexit()\nX_all = X_all[verified, :]\nidx = list(range(X_all.shape[0]))\nrandom.shuffle(idx)\nxSize = math.ceil(X_all.shape[0] / num_fold)\nsplit_X_path = os.path.join(base_path, 'X')\nsplit_y_path = os.path.join(base_path, 'y')\nif not os.path.exists(split_X_path):\n    os.makedirs(split_X_path)\nif not os.path.exists(split_y_path):\n    os.makedirs(split_y_path)\nfor i in range(num_fold):\n    X = X_all[idx[i*xSize:i*xSize+xSize]]\n    y = Y_all[idx[i*xSize:i*xSize+xSize]]\n    print('Saving fold {}'.format(i+1))\n    print('X.shape:', X.shape)\n    print('y.shape:', y.shape)\n    filename = os.path.join(split_X_path, 'X' + str(i+1) + '.npy')\n    np.save(filename, X)\n    filename = os.path.join(split_y_path, 'y' + str(i+1) + '.npy')\n    np.save(filename, y)\n    time.sleep(1)\n"

In [12]:
X_un.shape , X_ver.shape

((5763, 128, 63, 1), (3710, 128, 63, 1))

In [13]:
Y_ver = Y_ver['label'].map(map_dict)
Y_ver = np.array(Y_ver)
Y_un = Y_un['label'].map(map_dict)
Y_un = np.array(Y_un)
Y_un.shape , Y_ver.shape

((5763,), (3710,))

In [42]:
# enc = 
autoencoder.summary()
model = Model(input = autoencoder.layers[0].input, output = autoencoder.layers[5].output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 128, 63, 1)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 128, 63, 64)       1856      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 128, 63, 32)       57376     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 128, 63, 1)        897       
_________________________________________________________________
flatten_3 (Flatten)          (None, 8064)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 8064)              65036160  
_________________________________________________________________
reshape_3 (Reshape)          (None, 128, 63, 1)        0         
__________

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  app.launch_new_instance()


In [43]:
X_ver_dense = model.predict(X_ver)
X_un_dense = model.predict(X_un)
print(X_ver.shape , X_un.shape)

(3710, 128, 63, 1) (5763, 128, 63, 1)


In [45]:
X_test = np.load('feature/fbank2/X_test.npy')
X_test_dense = model.predict(X_test)

In [18]:
print(X_ver.shape, len(Y_ver))

(3710, 128, 63, 1) 3710


In [46]:
X_test_dense

array([[ 0.09620187,  0.2492431 ,  0.24759266, ...,  0.04647778,
        -0.02873544,  0.03930053],
       [ 0.09247909,  0.24535836,  0.24525653, ...,  0.04525119,
        -0.02469481,  0.03752303],
       [-0.3769405 , -1.7127558 , -0.18668665, ..., -0.6169543 ,
         0.46726173, -0.41674918],
       ...,
       [ 0.05762236,  0.24217416,  0.24650599, ...,  0.04710196,
        -0.00618235,  0.03439311],
       [-1.00396   ,  0.120268  ,  0.8185499 , ...,  0.02463336,
         0.09405316, -0.20394024],
       [ 0.09200265,  0.24472915,  0.24133763, ...,  0.04494386,
        -0.02631949,  0.03843198]], dtype=float32)

In [37]:
X_test_dense.shape

(9400, 8064)

In [47]:
lp_model = LabelSpreading(kernel='knn',n_jobs=-1)#,n_neighbors=3,alpha=0.05)#, max_iter=50, gamma=10, alpha=0.1,tol=0.0001)
lp_model.fit(X_ver_dense,Y_ver)

LabelSpreading(alpha=0.2, gamma=20, kernel='knn', max_iter=30, n_jobs=-1,
        n_neighbors=7, tol=0.001)

In [48]:
un_ans = lp_model.predict(X_un_dense)
test_ans = lp_model.predict(X_test_dense)
un_ans, test_ans

(array([ 0, 40,  7, ..., 15,  2, 39]), array([26, 39, 29, ...,  7, 21, 40]))

### Save semi data and re-label

In [50]:
print(X_un_dense.shape)

(5763, 8064)


In [53]:
np.save('feature/fbank2/semi/X_un_dense.npy',X_un_dense)
np.save('feature/fbank2/semi/Y_un_dense.npy',un_ans)
np.save('feature/fbank2/semi/X_test_dense.npy',X_test_dense)
np.save('feature/fbank2/semi/Y_test_dense.npy',test_ans)